# Búsqueda semántica
Autor: Eric S. Tellez <eric.tellez@infotec.mx>

Cuando la información esta poco específicada, las palabras adecuadas podrían ser dificiles de tener o limitar. En estos casos, la representaciones semánticas que permiten buscar _lo que se desea_ por medio de conceptos nos acerca más a la posibilidad de obtener información útil.

Como anteriormente se presento, una de las representaciones semánticas más aceptadas son aquellas basadas en word embeddings. Recordando, estas son representaciones vectoriales de cada palabra, donde la semántica se asocia con la estructura en un espacio métrico, i.e., cercano en la métrica significa similar semánticamente.

# Representación semántica de documentos basada en _word embeddings_
Un documento puede verse como una bolsa de palabras, como en el modelado tradicional, pero en lugar de usar los términos como símbolos se pueden usar los vectores semánticos. El como usar esta nube de puntos multidimensional para obtener una representación computacionalmente manejable y a la vez eficaz, es un tema que ha llevado al desarrollo de modelos cada vez más complejos, como pueden ser los grandes modelos de lenguaje. 

La manera forma más directa de definir una representación semántica de documentos es el uso de centroides, esto es, un vector de la misma dimensión que sumariza a un conjunto de vectores usando su media geométrica. De manera más precisa, sea $E$ una matrix $m \times n$ de embeddings del vocabulario, i.e., considerando un vocabulario de tamaño $n$ y dimensión $m$.
Sea $D_{m,\ell}$ la submatriz de $E$ que contiene los $\ell$ vectores del documento $\textsf{doc}$. El prototipo $\vec{d}$ de $D$ esta definido de la siguiente forma:

\begin{equation}
\vec{d} = \frac{1}{\ell} \sum_{i=1}^\ell D_i
\end{equation}


Donde $D_i$ es el i-ésimo vector columna de $D$. Dado que se usa el coseno como similitud, también es factible el cálculo como la suma vectorial normalizada, esto es,

$$ \vec{d} = \frac{\sum_{i=1}^\ell D_i}{\lVert \sum_{i=1}^\ell D_i \rVert}$$

También es posible añadir información local basada en la representación de bolsa de plabras. Por ejemplo, realizar una suma pesada usando el peso de las palabras mediante la frecuencia normalizada de término (TF) o la probabilidad de término $pt$, ver unidad 3 para más información.

$$\vec{d} = \sum_{i=1}^{\ell} \textsf{TF}(t_i, \textsf{doc}) \cdot D_i$$

Donde $t_i$ es el $i$-ésimo término de $\textsf{doc}$. Note que el orden no se captura y solo es necesario para la notación.

# Ejemplo


In [1]:
using Pkg
Pkg.activate(".")

using SimilaritySearch, TextSearch, Plots, KNearestCenters, LinearAlgebra, Embeddings, HypertextLiteral, CodecZlib, JSON, Base64, Random
using Downloads: download

  Activating project at `~/IR-2022/Unidades`


In [2]:
include("knn.jl")

scores (generic function with 1 method)

In [3]:
struct ValidVocabulary{DictType} <: AbstractTokenTransformation
    valid::DictType
end

TextSearch.transform_unigram(S::ValidVocabulary, tok) = haskey(S.valid, tok) ? tok : nothing

function text_model_and_vectors(
        corpus, vocab;
        localweighting=TpWeighting(),
        globalweighting=BinaryGlobalWeighting(),
        nlist=[1],
        qlist=[],
        slist=[],
        group_usr=false,
        group_url=true,
        group_num=true,
        del_diac=true,
        lc=true
    )

    voc = Vocabulary(length(corpus))
    for v in vocab
       push!(voc, v)
    end
    
    tt = ValidVocabulary(voc.token2id)
    textconfig = TextConfig(; group_usr, group_url, del_diac, lc, group_num, nlist, qlist, slist, tt)
    tokenize_and_append!(voc, textconfig, corpus)
    model = VectorModel(globalweighting, localweighting, voc)
    (; textconfig, model, voc)
end


text_model_and_vectors (generic function with 1 method)

In [13]:
function embeddings(embname="MX.vec")
    embfile = "../data/$embname"
    !isfile(embfile) && download("http://geo.ingeotec.mx/~sadit/regional-spanish-models/$embname", embfile)
    emb = load_embeddings(FastText_Text, embfile)  # you can change with any of the available embeddings in `Embeddings`
    #=for c in eachcol(emb.embeddings)
        normalize!(c)
    end=#
    
    (; X=emb.embeddings, vocab=emb.vocab)
end

function vectorize_as_prototype!(c, text, E, T)
    x = vectorize(T.model, T.textconfig, text; normalize=false)
    if length(x) == 1 && haskey(x, 0)
        #@warn "empty vector $(Int(i)) selecting a random vector for it " # $(corpus[i])
        rand!(c)
    else
        for (id, weight) in x
            id == 0 && continue
             c .= c .+ weight .* view(E.X, :, id)
        end
    end
    
    #normalize!(c)
    c
end

function vectorize_corpus_as_prototypes(corpus, E, T)
    dim = size(E.X, 1)
    n = length(corpus)
    C = zeros(Float32, dim, n)
    # Threads.@threads
    for i in 1:n
        vectorize_as_prototype!(view(C, :, i), corpus[i], E, T)
    end
    
    C
end

function create_index(db, recall)
    dist = SqL2Distance()
    index = SearchGraph(; dist, db, verbose=false)
    index!(index; callbacks=SearchGraphCallbacks(MinRecall(recall)))
    optimize!(index, MinRecall(recall))
    index
end

create_index (generic function with 1 method)

In [5]:
@time E = embeddings()

 33.656451 seconds (7.49 M allocations: 11.543 GiB, 2.44% gc time, 0.87% compilation time)


(X = Float32[-0.07049 -0.11775 … -0.14671 0.173; 0.16225 0.10458 … -0.05029 0.20899; … ; -0.0786 0.12893 … -0.1113 -0.31056; 0.03832 0.03368 … -0.00493 0.0213], vocab = ["</s>", "_usr", "que", "de", ",", ".", "y", "a", "la", "no"  …  "tannehil", "movelike", "peliblanca", "cristianzav", "rqq", "preguntartee", "sutedes", "venciendolo", "eviscerado", "fectivo"])

In [6]:
display(@htl "<h1>Cargando el corpus</h1>")

include("read_datasets.jl")
D, Q = read_news()
T = text_model_and_vectors(D.corpus, E.vocab)
@show unique(D.labels), T.model

<h1>Cargando el corpus</h1>

(unique(D.labels), T.model) = (["AdriDelgadoRuiz", "El_Universal_Mx", "CNNEE", "NTN24", "UniNoticias", "TelemundoNews", "SinEmbargoMX", "Reforma", "abc_es", "azucenau", "AristeguiOnline", "el_pais", "EFEnoticias", "PublimetroMX", "PublimetroChile", "Radio_Formula", "RicardoAlemanMx", "epigmenioibarra", "Milenio", "LaRazon_mx", "abrahamendieta", "PublimetroCol", "teleSURtv", "bbcmundo", "julioastillero"], {VectorModel global_weighting=BinaryGlobalWeighting(), local_weighting=TpWeighting(), train-voc=438136, train-n=30244, maxoccs=43239})


(["AdriDelgadoRuiz", "El_Universal_Mx", "CNNEE", "NTN24", "UniNoticias", "TelemundoNews", "SinEmbargoMX", "Reforma", "abc_es", "azucenau"  …  "Radio_Formula", "RicardoAlemanMx", "epigmenioibarra", "Milenio", "LaRazon_mx", "abrahamendieta", "PublimetroCol", "teleSURtv", "bbcmundo", "julioastillero"], {VectorModel global_weighting=BinaryGlobalWeighting(), local_weighting=TpWeighting(), train-voc=438136, train-n=30244, maxoccs=43239})

In [7]:
@time C = vectorize_corpus_as_prototypes(D.corpus, E, T)

  1.542268 seconds (3.34 M allocations: 312.757 MiB, 29.45% compilation time)


300×30244 Matrix{Float32}:
 -0.0716944   -0.0126372    -0.0639624   …  -0.114824    -0.164547
  0.0222967    0.00466333   -0.00958138      0.0400896    0.0723063
  0.00139444   0.0134513    -0.00457931     -0.00827696   0.0358446
 -0.08063     -0.0221579    -0.02271         0.0568417    0.0490838
  0.0766478    0.000928206   0.00133828     -0.0530687   -0.0109383
  0.110559    -0.0319262     0.048371    …   0.00874826  -0.00037667
 -0.0803789    0.0292315     0.0357379       0.00898913   0.0120629
  0.0959622    0.0295805    -0.0130335      -0.0664948    0.0322692
  0.114802     0.0164505     0.126971        0.149568     0.185204
  0.0238333   -0.0266008    -0.00980345     -0.0192987   -0.0253933
  0.121443     0.0632185     0.13941     …   0.136218     0.166292
  0.134657    -0.0398746     0.0664055       0.044667     0.0862833
  0.116133    -0.00271231    0.0980417       0.12451      0.0429154
  ⋮                                      ⋱               
 -0.0380811   -0.12427      -0.08

### Se crea el índice métrico

In [14]:
@time index = create_index(MatrixDatabase(C), 0.95)

 20.719137 seconds (4.72 M allocations: 676.241 MiB, 2.31% gc time, 27.03% compilation time)


SearchGraph{SqL2Distance, MatrixDatabase{Matrix{Float32}}, BeamSearch}
  dist: SqL2Distance SqL2Distance()
  db: MatrixDatabase{Matrix{Float32}}
  links: Array{Vector{Int32}}((30244,))
  locks: Array{Base.Threads.SpinLock}((30244,))
  hints: Array{Int32}((105,)) Int32[42, 101, 107, 284, 315, 354, 466, 500, 551, 577  …  2300, 2329, 2344, 2363, 2366, 2370, 2386, 2401, 2440, 25642]
  search_algo: BeamSearch
  verbose: Bool false


### Búsqueda de todos los vecinos cercanos en el vocabulario, observe la conveniencia del uso de un índice

Entre más sean las consultas más se ve la bondad (~42k consultas)

In [15]:
let k = 11
    Qvectors = MatrixDatabase(vectorize_corpus_as_prototypes(Q.corpus, E, T))
    t1 = @elapsed I, _ = searchbatch(index, Qvectors, k)
    # WARNING: Don't run the following line, it takes too much time
    ex = ExhaustiveSearch(; db=index.db, dist=index.dist)
    t2 = @elapsed gI, _ = searchbatch(ex, Qvectors, k)
    r = macrorecall(gI, I)
    n = size(I, 2)
    labels = String.(D.labels)
    s1 = scores(Q.labels, knn(I, labels))
    s2 = scores(Q.labels, knn(gI, labels))
    
    @htl """
    <div>(brute vs indexed) macro-recall: $r, n: $n</div>
    <div>searchgraph search time: $t1, scores: $s1</div>
    <div>brute force search time: $t2, scores: $s2</div>
    """
end

<div>(brute vs indexed) macro-recall: 0.9334613616077625, n: 3361</div>
<div>searchgraph search time: 0.540023268, scores: <span class="Core-NamedTuple">(macrof1 = 0.26325391828720446, macrorecall = 0.2585745359480052, accuracy = 0.28324903302588517)</span></div>
<div>brute force search time: 1.125471385, scores: <span class="Core-NamedTuple">(macrof1 = 0.27446725406455935, macrorecall = 0.27056569458624175, accuracy = 0.29604284439155015)</span></div>

### Búsqueda y presentación de los resultados

In [10]:
function search_and_display(index, qtext, k, D, E, T)
    res = KnnResult(k)
    q = zeros(Float32, size(E.X, 1))
    vectorize_as_prototype!(q, qtext, E, T)
    @time search(index, q, res)
    
    L = []
    for (j, (id, d)) in enumerate(res)
        push!(L, @htl "<tr><td>$j</td><td>$id</td><td>$(round(d, digits=3))</td> <td>$(D.labels[id])</td><td> $(D.corpus[id])</td> </tr>")
    end

    display(@htl """<h2>resultados for "$qtext"</h2>
    <table>
    <th>  <td>id</td> <td>dist</td> <td>user</td> </td>message<td> </th>
        $L
    </table>
    """)
end


search_and_display (generic function with 1 method)

In [11]:

display(@htl "<h1>Ejemplos de búsqueda</h1>")
search_and_display(index, "el gobierno de andres manuel lopez", 7, D, E, T)
search_and_display(index, "trafico de drogas", 7, D, E, T)
search_and_display(index, "covid corona virus", 7, D, E, T)


<h1>Ejemplos de búsqueda</h1>

  0.000648 seconds


1,18286,-3.006,RicardoAlemanMx,AMLO arremetió contra Calderón https://t.co/h6eMNroqT1
2,3296,-2.719,LaRazon_mx,Descarta el presidente @lopezobrador_ renuncia del fiscal Alejandro Gertz Manero https://t.co/NCrXYcXYKF https://t.co/jaLL68tlRs
3,2292,-2.635,El_Universal_Mx,RT @ElUniversal_SLP: #Entérate El gobernador Ricardo Gallardo Cardona pidió al presidente Andrés Manuel López Obrador apoyo para reforzar l…
4,24486,-2.595,Radio_Formula,Felipe Fuentes Barrera es nombrado presidente interino del TEPJF https://t.co/KO00PbeSbN https://t.co/jxnfe2bN2U
5,6262,-2.583,LaRazon_mx,🚨 #ÚLTIMAHORA | Reyes Rodríguez Mondragón es electo magistrado presidente del TEPJF https://t.co/uDL9CIupaz https://t.co/JTBU4SCdSf
6,24904,-2.412,azucenau,"#AzucenaxMilenio | Durante #LaMañanera, el presidente López Obrador respaldó nuevamente al gobernador de Veracruz, Cuitláhuac García https://t.co/iX2ubBHbyJ"
7,1231,-2.388,azucenau,"#ENVIVO | Mensaje conjunto del presidente López Obrador y Alejandro Giammattei, presidente de #Guatemala https://t.co/uOwcCHGKt3"


  0.000744 seconds


1,4059,-2.259,El_Universal_Mx,Regresan los asaltos a automovilistas en Periférico ▶️ https://t.co/5j02oCFXHZ
2,16919,-2.169,LaRazon_mx,. @ONU_es aprueba resolución mexicana para combatir tráfico de armas https://t.co/B7YEbjID5n https://t.co/WBD30vdeJa
3,29911,-2.131,Milenio,▶ Choferes de tráileres golpean a agentes de tránsito tras intentar multarlos https://t.co/xnMf2Yqvcv https://t.co/UT5XyxGp6F
4,30159,-2.121,AristeguiOnline,Decomisan metanfetamina oculta en dulces de palanqueta en Sinaloa https://t.co/onlqnv3Fd5 https://t.co/JSRR9wYxco
5,12829,-2.092,Radio_Formula,"Insurgentes Norte, Calzada Ignacio Zaragoza y hasta Tlalpan presentan tránsito fluido en ambos sentidos. https://t.co/QF5BfhSgMP"
6,24701,-1.978,AristeguiOnline,Militares decomisan 200 kilos de cocaína en Sonora y SLP https://t.co/HJpfqx4Jya https://t.co/VlXVvHtpDA
7,4489,-1.978,PublimetroMX,▶️ #VIDEO l Camioneta de carga provoca carambola en avenida de #Toluca https://t.co/F4zCYXsMav


  0.000050 seconds


1,163,-1.619,SinEmbargoMX,¿Un chicle contra la COVID? Crean producto que puede frenar la transmisión del virus https://t.co/L0r3yXu4yA https://t.co/qM4VVEuaEi
2,1600,-1.492,AristeguiOnline,#Entérate | Anuncia López-Gatell vacuna de refuerzo contra Covid solo para adultos mayores 👉 https://t.co/q1vRlO8QCG https://t.co/uzHk1zXzNP
3,2303,-1.481,Radio_Formula,Gibran Lajud obtiene pasaporte para poder ser convocado a la selección de Líbano https://t.co/8l43rOPWMi https://t.co/VlVajoYIto
4,2015,-1.46,el_pais,RT @elpais_tec: La ciberguerra de Rusia contra Ucrania nunca ha acabado https://t.co/vjzJKkFAke
5,1591,-1.444,AristeguiOnline,#Video ▶️| Champions League: Asistirá el Rey Felipe VI a la Final en París https://t.co/3H0LCMv92q
6,1479,-1.405,EFEnoticias,"Millones de toneladas de desechos sanitarios, otro precio de la pandemia. https://t.co/cYgWsRhnzf https://t.co/F8ozpFPj9M"
7,1670,-1.395,AristeguiOnline,Shanghái ve próximo el fin del confinamiento tras brote de Covid-19 https://t.co/PKCeJiQttL


In [12]:
display(@htl "<h1>Ejemplos de búsqueda (mensajes aleatorios)</h1>")

for i in 1:3
    for qid in rand(1:length(D.corpus))
        search_and_display(index, D.corpus[qid], 7, D, E, T)
    end
end

<h1>Ejemplos de búsqueda (mensajes aleatorios)</h1>

  0.003390 seconds


1,6854,-1.107,PublimetroMX,RT @Publisport_MX: #RepechajeUEFA ¡Comienza el partido! #Escocia 🏴󠁧󠁢󠁳󠁣󠁴󠁿 0-0 🇺🇦 #Ucrania https://t.co/jcH7NIt9M0 https://t.co/yxOuBPi…
2,2483,-1.086,LaRazon_mx,🚨 #ÚLTIMAHORA | SCJN avala aborto a menores de 12 a 17 años víctimas de violación https://t.co/I9c1D4FZop https://t.co/ioa3Etu4aD
3,22625,-1.08,LaRazon_mx,"#OJO | Autoridades minimizan violencia contra las mujeres, afirman activistas https://t.co/YQHOHwtURp https://t.co/kfFx5tQMdB"
4,23958,-1.071,Milenio,Dictan prisión preventiva oficiosa a presunto asesino de María Fernanda https://t.co/OCPZI8NksD https://t.co/sWp0ImFxem
5,10619,-1.063,LaRazon_mx,🚨 #ÚLTIMAHORA | Aprueba @SCJN despenalizar aborto; determina que mujeres decidan interrupción del embarazo https://t.co/b6asxzys76 https://t.co/SnYMKxtm0B
6,2570,-1.054,AristeguiOnline,#Video Bachelet denuncia agresiones contra ambientalistas en América Latina https://t.co/9AqXtIgLpu
7,8523,-1.051,AristeguiOnline,Condenan organismos de derechos humanos el asesinato de Lourdes Maldonado https://t.co/Mp151TrWWx https://t.co/VZLcRkfC3x


  0.002716 seconds


1,23936,-1.365,El_Universal_Mx,🚨 #ÚltimaHora Reportan cuatro muertos tras balacera durante operativo en Azcapotzalco https://t.co/Sh6GiJxAJc
2,7866,-1.272,julioastillero,Reporte de @ArturoCanoMx
3,19519,-1.257,El_Universal_Mx,🚨#ÚltimaHora🚨 Pemex restablece producción de barriles tras explosión en plataforma https://t.co/P0rcDlMnl9 https://t.co/xnye3Ud3dj
4,6854,-1.246,PublimetroMX,RT @Publisport_MX: #RepechajeUEFA ¡Comienza el partido! #Escocia 🏴󠁧󠁢󠁳󠁣󠁴󠁿 0-0 🇺🇦 #Ucrania https://t.co/jcH7NIt9M0 https://t.co/yxOuBPi…
5,28378,-1.208,LaRazon_mx,Suman 4 detenidos tras hallazgo de tráiler con migrantes muertos en Texas https://t.co/HJx4W93IxU https://t.co/ZBwnDsndOG
6,22847,-1.197,El_Universal_Mx,🚨 #ÚltimaHora 🚨 Enfrentamientos entre civiles armados han dejado al menos dos policías estatales heridos en Tamaulipas https://t.co/WK9Q5dowXO
7,13538,-1.16,julioastillero,Cinco gobernadores negocian en EU https://t.co/XF3mBKhfU0


  0.003064 seconds


1,18286,-1.568,RicardoAlemanMx,AMLO arremetió contra Calderón https://t.co/h6eMNroqT1
2,6854,-1.464,PublimetroMX,RT @Publisport_MX: #RepechajeUEFA ¡Comienza el partido! #Escocia 🏴󠁧󠁢󠁳󠁣󠁴󠁿 0-0 🇺🇦 #Ucrania https://t.co/jcH7NIt9M0 https://t.co/yxOuBPi…
3,7212,-1.426,LaRazon_mx,.@SandraCuevas_ promete desmantelar cárteles en la alcaldía #Cuauhtémoc https://t.co/XnCpQv0IXd https://t.co/VeyuhEIbbp
4,2570,-1.381,AristeguiOnline,#Video Bachelet denuncia agresiones contra ambientalistas en América Latina https://t.co/9AqXtIgLpu
5,6262,-1.379,LaRazon_mx,🚨 #ÚLTIMAHORA | Reyes Rodríguez Mondragón es electo magistrado presidente del TEPJF https://t.co/uDL9CIupaz https://t.co/JTBU4SCdSf
6,4673,-1.356,AristeguiOnline,Vandalizan instalaciones del Instituto Estatal Electoral en Oaxaca https://t.co/DxoSaCG7Ag
7,6054,-1.353,LaRazon_mx,AMLO defiende en Sinaloa estrategia contra el crimen https://t.co/eGmxU1Cffz https://t.co/nTjcE7Squa


# Actividades
- Reproduzca el ejercicio de este notebook, use embeddings para español, cambié los ejemplos. Se sugiere el uso de <https://ingeotec.github.io/regional-spanish-models/> donde encontrará modelos fastText regionalizados del español, pero puede usar otros embeddings.
- ¿Qué piensa de las diferencias de tamaño entre los documentos y las consultas? esto como afecta a la representación semántica.
- ¿Cuál sería el símil de bigramas y trigramas para este esquema de representación semántica? Implementelo.
- Implemente su búsqueda semántica. Si usa Julia considere `SimilaritySearch.jl` y si usa Python considere `faiss`.
- Reporte su notebook y anote sus soluciones a las preguntas planteadas. Reporte los resultados de sus implementaciones, compare contra las alternativas presentadas en este reporte. Discuta sus resultados. Finalice el reporte  con reflexiones sobre el uso de nubes de puntos en lugar de bolsas de palabras tradicionales. Anoté sus conclusiones.

# Bibliografía
- [KSKW2015] Kusner, M., Sun, Y., Kolkin, N., & Weinberger, K. (2015, June). From word embeddings to document distances. In International conference on machine learning (pp. 957-966). PMLR.
- [PW2009] Pele, O., & Werman, M. (2009, September). Fast and robust earth mover's distances. In 2009 IEEE 12th international conference on computer vision (pp. 460-467). IEEE.
